In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed

# Función para solicitar y obtener el contenido de una página
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

# Función para completar una URL relativa
def complete_url(relative_url):
    if not relative_url.startswith('http'):
        return f'https://sitllx.diputados.gob.mx/{relative_url}'
    return relative_url

# Función para extraer la información académica
def extract_education(soup):
    education_data = []
    education_rows = soup.find_all('tr', class_='textoNegro')
    for row in education_rows:
        cells = row.find_all('td')
        if len(cells) == 3:  # Grado, campo de estudio, años
            education_data.append(f"{cells[0].text.strip()}: {cells[1].text.strip()} ({cells[2].text.strip()})")
    return ' | '.join(education_data)

# Función para extraer los enlaces a las iniciativas políticas
def extract_policy_links(soup):
    policy_links = soup.select('td.Estilo51 a.linkBlancoSin')
    return [complete_url(a['href']) for a in policy_links]

# Función para extraer las políticas de cada periodo activo
def extract_policies(policy_links):
    policies = []
    for link in policy_links:
        policies_page = fetch_page(link)
        policies.extend(extract_policy_data(policies_page))
    return ' | '.join(policies)

# Función para extraer la información de las políticas de cada enlace
def extract_policy_data(policy_soup):
    policy_data = []
    policy_rows = policy_soup.select('td.Estilo69 span.Estilo71')
    for policy_row in policy_rows:
        policy_text = policy_row.get_text(strip=True)
        if policy_text:
            policy_data.append(policy_text)
    return policy_data

# Función para extraer la información de cada diputado
def extract_deputy_info(deputy_url):
    soup = fetch_page(deputy_url)
    education = extract_education(soup)
    policy_links = extract_policy_links(soup)
    policies = extract_policies(policy_links)
    return education, policies

# URL inicial con la lista de diputados
base_url = 'https://sitllx.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Obtener la lista de diputados y sus URLs
deputies_list = [(dep.text, complete_url(dep['href'])) for dep in fetch_page(base_url).select('td > a.linkVerde')]

# Usar ThreadPoolExecutor para extraer la información de manera concurrente
deputies_info = []

with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_deputy = {executor.submit(extract_deputy_info, url): name for name, url in deputies_list}
    for future in concurrent.futures.as_completed(future_to_deputy):
        deputy_name = future_to_deputy[future]
        try:
            education, policies = future.result()
            deputies_info.append({
                'Name': deputy_name,
                'Education': education,
                'Policies': policies
            })
        except Exception as e:
            print(f"Error processing {deputy_name}: {e}")

# Crear un DataFrame con la información extraída
df = pd.DataFrame(deputies_info)

print(df)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Función para realizar la solicitud y obtener el contenido de una página
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

# Función para completar una URL relativa
def complete_url(relative_url):
    if not relative_url.startswith('http'):
        return f'https://sitllx.diputados.gob.mx/{relative_url}'
    return relative_url

# Función para extraer la información académica
def extract_education(soup):
    education_data = []
    # Se busca la sección de escolaridad basada en el título de la sección
    education_section = soup.find('td', class_='TitulosVerde', string='ESCOLARIDAD')
    if education_section:
        # Se buscan todos los elementos 'tr' después del título de escolaridad
        education_rows = education_section.find_all_next('tr', limit=3)  # Asumimos que hay 3 filas de escolaridad
        for row in education_rows:
            # Se extrae el texto de cada celda 'td' dentro del 'tr' y se ignora la primera celda vacía
            education_data.append(' '.join(td.get_text(strip=True) for td in row.find_all('td', class_='textoNegro')[1:]))
    return ' | '.join(education_data)

# Función para extraer los enlaces a las iniciativas políticas
def extract_policy_links(soup):
    return [complete_url(a['href']) for a in soup.select('td.Estilo51 a.linkBlancoSin')]

# Función para extraer la información de las políticas propuestas
def extract_policies(policy_links):
    policies_data = []
    for link in policy_links:
        policy_page_soup = fetch_page(link)
        # Se extraen las descripciones de las políticas de las celdas específicas
        policies_rows = policy_page_soup.select('table[valign="top"] > tr > td[valign="top"].Estilo69')
        for row in policies_rows:
            policy_description = row.select_one('span.Estilo71')
            if policy_description:
                policies_data.append(policy_description.get_text(strip=True))
    return ' | '.join(policies_data)

# Función para extraer la información de cada diputado
def extract_deputy_info(deputy_url):
    soup = fetch_page(deputy_url)
    education = extract_education(soup)
    policy_links = extract_policy_links(soup)
    policies = extract_policies(policy_links)
    return {'Education': education, 'Policies': policies}

# URL inicial con la lista de diputados
base_url = 'https://sitllx.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Obtener la lista de diputados y sus URLs
deputies_list = [(dep.text, complete_url(dep['href'])) for dep in fetch_page(base_url).select('td > a.linkVerde')]

# Configurar el ThreadPoolExecutor para extraer la información de manera concurrente
deputies_info = []

with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_deputy = {executor.submit(extract_deputy_info, deputy_data[1]): deputy_data[0] for deputy_data in deputies_list}
    for future in concurrent.futures.as_completed(future_to_deputy):
        deputy_name = future_to_deputy[future]
        try:
            deputy_info = future.result()
            deputies_info.append({
                'Name': deputy_name,
                **deputy_info
            })
        except Exception as e:
            print(f"Error processing {deputy_name}: {e}")

# Crear DataFrame de pandas con la información extraída
df = pd.DataFrame(deputies_info)
print(df)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Función para realizar la solicitud y obtener el contenido de una página
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

# Función para completar una URL relativa
def complete_url(relative_url):
    if not relative_url.startswith('http'):
        return f'https://sitllx.diputados.gob.mx/{relative_url}'
    return relative_url

# Función para extraer la información académica
def extract_education(soup):
    education_data = []
    # Se busca la sección de escolaridad basada en el título de la sección
    education_section = soup.find('td', class_='TitulosVerde', string='ESCOLARIDAD')
    if education_section:
        # Se buscan todos los elementos 'tr' después del título de escolaridad
        education_rows = education_section.find_all_next('tr', limit=3)  # Asumimos que hay 3 filas de escolaridad
        for row in education_rows:
            # Se extrae el texto de cada celda 'td' dentro del 'tr' y se ignora la primera celda vacía
            education_data.append(' '.join(td.get_text(strip=True) for td in row.find_all('td', class_='textoNegro')[1:]))
    return ' | '.join(education_data)

# Función para extraer los enlaces a las iniciativas políticas
def extract_policy_links(soup):
    return [complete_url(a['href']) for a in soup.select('td.Estilo51 a.linkBlancoSin')]

# Función para extraer la información de las políticas propuestas
def extract_policies(policy_links):
    policies_data = []
    for link in policy_links:
        policy_page_soup = fetch_page(link)
        # Se corrige el selector CSS basado en la estructura HTML proporcionada
        policy_descriptions = policy_page_soup.select('td[valign="top"].Estilo69 span.Estilo71')
        for policy in policy_descriptions:
            # Extraer solo el texto de las políticas, sin el número y suscriptor
            if policy.find('b') is not None:  # Si hay un elemento en negrita (número de la política), se ignora
                policy_text = policy.get_text(separator=' ', strip=True)
                policies_data.append(policy_text)
    return ' | '.join(policies_data)

# Función para extraer la información de cada diputado
def extract_deputy_info(deputy_url):
    soup = fetch_page(deputy_url)
    education = extract_education(soup)
    policy_links = extract_policy_links(soup)
    policies = extract_policies(policy_links)
    return {'Education': education, 'Policies': policies}

# URL inicial con la lista de diputados
base_url = 'https://sitllx.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Obtener la lista de diputados y sus URLs
deputies_list = [(dep.text, complete_url(dep['href'])) for dep in fetch_page(base_url).select('td > a.linkVerde')]

# Configurar el ThreadPoolExecutor para extraer la información de manera concurrente
deputies_info = []

with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_deputy = {executor.submit(extract_deputy_info, deputy_data[1]): deputy_data[0] for deputy_data in deputies_list}
    for future in concurrent.futures.as_completed(future_to_deputy):
        deputy_name = future_to_deputy[future]
        try:
            deputy_info = future.result()
            deputies_info.append({
                'Name': deputy_name,
                **deputy_info
            })
        except Exception as e:
            print(f"Error processing {deputy_name}: {e}")

# Crear DataFrame de pandas con la información extraída
df = pd.DataFrame(deputies_info)
print(df)

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# URL de la página con la lista de diputados
url_base = 'https://sitllx.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Función para obtener el objeto BeautifulSoup de una página de forma concurrente
def obtener_html(url):
    with requests.Session() as session:  # Utiliza una sesión para reutilizar la conexión
        respuesta = session.get(url)
        respuesta.raise_for_status()  # Lanzar error si la solicitud falla
        return BeautifulSoup(respuesta.content, 'html.parser')

# Función para extraer la lista de diputados y sus URLs
def extraer_lista_diputados(soup):
    diputados = []
    for enlace in soup.select('a.linkVerde'):
        nombre = enlace.get_text(strip=True)
        url_diputado = 'https://sitllx.diputados.gob.mx/' + enlace['href']
        diputados.append({'nombre': nombre, 'url': url_diputado})
    return diputados

# Función para extraer la información educativa de la página de un diputado
def extraer_informacion_educativa(soup):
    educacion = []
    seccion_escolaridad = soup.find('td', text='ESCOLARIDAD')
    # Verificar que se encontró la sección antes de continuar
    if seccion_escolaridad:
        tabla_escolaridad = seccion_escolaridad.find_next('table')
        if tabla_escolaridad:
            for fila in tabla_escolaridad.find_all('tr'):
                celdas = fila.find_all('td')
                if celdas and len(celdas) == 3:
                    grado, campo, periodo = [celda.get_text(strip=True) for celda in celdas]
                    educacion.append({'grado': grado, 'campo': campo, 'periodo': periodo})
    return educacion

# Función para extraer los enlaces a las iniciativas de un diputado
def extraer_enlaces_iniciativas(soup):
    enlace_iniciativas = soup.find('a', text='Iniciativas')
    if enlace_iniciativas:
        url_iniciativas = 'https://sitllx.diputados.gob.mx/' + enlace_iniciativas['href']
        soup_iniciativas = obtener_html(url_iniciativas)
        enlaces_periodos = soup_iniciativas.select('a.linkVerde')
        iniciativas = [extraer_texto_iniciativas('https://sitllx.diputados.gob.mx/' + enlace['href']) for enlace in enlaces_periodos]
        return iniciativas
    return []

# Función para extraer el texto de las iniciativas de un periodo de sesiones
def extraer_texto_iniciativas(url_periodo):
    soup_periodo = obtener_html(url_periodo)
    textos = []
    for iniciativa in soup_periodo.select('span.Estilo71'):
        texto = iniciativa.get_text(strip=True)
        if texto:
            textos.append(texto)
    return textos

# Función para obtener la información de un diputado
def obtener_info_diputado(url):
    soup_diputado = obtener_html(url)
    nombre = soup_diputado.find('span', class_='Estilo67').text.strip()
    educacion = extraer_informacion_educativa(soup_diputado)
    iniciativas = extraer_enlaces_iniciativas(soup_diputado)
    return {
        'nombre': nombre,
        'educacion': educacion,
        'iniciativas': iniciativas
    }

# Función principal para extraer la información de todos los diputados
def extraer_informacion_diputados_concurrent():
    soup_lista = obtener_html(url_base)
    lista_diputados = extraer_lista_diputados(soup_lista)
    urls_diputados = [diputado['url'] for diputado in lista_diputados]
    
    datos_diputados = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(obtener_info_diputado, url): url for url in urls_diputados}
        for future in as_completed(future_to_url):
            try:
                datos_diputados.append(future.result())
            except Exception as exc:
                print(f'Hubo un problema con la URL {future_to_url[future]}: {exc}')
    
    return pd.DataFrame(datos_diputados)

# Ejecutar la función principal y mostrar/guardar el DataFrame resultante
df_diputados = extraer_informacion_diputados_concurrent()
print(df_diputados)

                                         nombre educacion  \
0           Dip. Martía Teresa Alanís Domínguez        []   
1    Dip. María del Consuelo Argüelles Arellano        []   
2                  Dip. Gerardo Amezola Fonceca        []   
3                  Dip. Pedro Armendáriz García        []   
4                    Dip. Gerardo Aranda Orozco        []   
..                                          ...       ...   
495                  Dip. Armando García Méndez        []   
496               Dip. Humberto López Lena Cruz        []   
497      Dip. Elsa de Guadalupe Conde Rodríguez        []   
498          Dip. Santiago Gustavo Pedro Cortés        []   
499               Dip. Aida Marina Arvizu Rivas        []   

                                           iniciativas  
0                                                   []  
1    [[1Proyecto de decreto que reforma el Artículo...  
2    [[1Proyecto de decreto que reforma los artícul...  
3    [[1Proyecto de decreto que reforma

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# URL de la página con la lista de diputados
url_base = 'https://sitllx.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Función para obtener el objeto BeautifulSoup de una página de forma concurrente
def obtener_html(url):
    with requests.Session() as session:  # Utiliza una sesión para reutilizar la conexión
        respuesta = session.get(url)
        respuesta.raise_for_status()  # Lanzar error si la solicitud falla
        return BeautifulSoup(respuesta.content, 'html.parser')

# Función para extraer la lista de diputados y sus URLs
def extraer_lista_diputados(soup):
    diputados = []
    for enlace in soup.select('a.linkVerde'):
        nombre = enlace.get_text(strip=True)
        url_diputado = 'https://sitllx.diputados.gob.mx/' + enlace['href']
        diputados.append({'nombre': nombre, 'url': url_diputado})
    return diputados

# Función para extraer la información educativa de la página de un diputado
def extraer_informacion_educativa(soup):
    educacion = []
    # Encuentra todas las entradas después del encabezado 'ESCOLARIDAD'
    seccion_escolaridad = soup.find('td', class_='TitulosVerde', text='ESCOLARIDAD')
    if seccion_escolaridad:
        # La sección de escolaridad está en la misma tabla, pero después del título 'ESCOLARIDAD'
        entradas_escolaridad = seccion_escolaridad.find_all_next('tr')
        for entrada in entradas_escolaridad:
            celdas = entrada.find_all('td', class_='textoNegro')
            # Podría haber filas que no pertenecen a la sección escolaridad, como 'TRAYECTORIA POLÍTICA'
            # Así que verifica si la fila pertenece a la sección escolaridad
            if celdas and len(celdas) >= 2:  # Asumiendo que hay al menos dos celdas (grado y campo)
                # Extraer el grado, campo y periodo si están presentes
                grado = celdas[0].get_text(strip=True) if len(celdas) > 0 else ''
                campo = celdas[1].get_text(strip=True) if len(celdas) > 1 else ''
                periodo = celdas[2].get_text(strip=True) if len(celdas) > 2 else ''
                educacion.append({'grado': grado, 'campo': campo, 'periodo': periodo})
            # Si se encuentra un nuevo encabezado, se detiene la extracción
            if entrada.find('td', class_='TitulosVerde'):
                break
    return educacion

# Función para extraer los enlaces a las iniciativas de un diputado
def extraer_enlaces_iniciativas(soup):
    enlace_iniciativas = soup.find('a', text='Iniciativas')
    if enlace_iniciativas:
        url_iniciativas = 'https://sitllx.diputados.gob.mx/' + enlace_iniciativas['href']
        soup_iniciativas = obtener_html(url_iniciativas)
        enlaces_periodos = soup_iniciativas.select('a.linkVerde')
        iniciativas = [extraer_texto_iniciativas('https://sitllx.diputados.gob.mx/' + enlace['href']) for enlace in enlaces_periodos]
        return iniciativas
    return []

# Función para extraer el texto de las iniciativas de un periodo de sesiones
def extraer_texto_iniciativas(url_periodo):
    soup_periodo = obtener_html(url_periodo)
    textos = []
    for iniciativa in soup_periodo.select('span.Estilo71'):
        texto = iniciativa.get_text(strip=True)
        if texto:
            textos.append(texto)
    return textos

# Función para obtener la información de un diputado
def obtener_info_diputado(url):
    soup_diputado = obtener_html(url)
    nombre = soup_diputado.find('span', class_='Estilo67').text.strip()
    educacion = extraer_informacion_educativa(soup_diputado)
    iniciativas = extraer_enlaces_iniciativas(soup_diputado)
    return {
        'nombre': nombre,
        'educacion': educacion,
        'iniciativas': iniciativas
    }

# Función principal para extraer la información de todos los diputados
def extraer_informacion_diputados_concurrent():
    soup_lista = obtener_html(url_base)
    lista_diputados = extraer_lista_diputados(soup_lista)
    urls_diputados = [diputado['url'] for diputado in lista_diputados]
    
    datos_diputados = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(obtener_info_diputado, url): url for url in urls_diputados}
        for future in as_completed(future_to_url):
            try:
                datos_diputados.append(future.result())
            except Exception as exc:
                print(f'Hubo un problema con la URL {future_to_url[future]}: {exc}')
    
    return pd.DataFrame(datos_diputados)

# Ejecutar la función principal y mostrar/guardar el DataFrame resultante
df_diputados = extraer_informacion_diputados_concurrent()
print(df_diputados)

Hubo un problema con la URL https://sitllx.diputados.gob.mx/curricula.php?dipt=262: HTTPSConnectionPool(host='sitllx.diputados.gob.mx', port=443): Max retries exceeded with url: /iniciativas_por_pernp.php?iddipt=262&pert=5 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E853360810>, 'Connection to sitllx.diputados.gob.mx timed out. (connect timeout=None)'))
Hubo un problema con la URL https://sitllx.diputados.gob.mx/curricula.php?dipt=455: HTTPSConnectionPool(host='sitllx.diputados.gob.mx', port=443): Max retries exceeded with url: /iniciativas_por_pernp.php?iddipt=455&pert=6 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E8546D1450>, 'Connection to sitllx.diputados.gob.mx timed out. (connect timeout=None)'))
Hubo un problema con la URL https://sitllx.diputados.gob.mx/curricula.php?dipt=434: HTTPSConnectionPool(host='sitllx.diputados.gob.mx', port=443): Max retries exceeded with url: /iniciativas_por_pernp.php?i

In [8]:
# Suponiendo que 'df_diputados' es tu DataFrame

# Convertir el DataFrame a CSV
nombre_archivo = "diputados.csv"
df_diputados.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

print(f"Archivo guardado como: {nombre_archivo}")

Archivo guardado como: diputados.csv


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# URL de la página con la lista de diputados
url_base = 'https://sitllxiii.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Función para obtener el objeto BeautifulSoup de una página de forma concurrente
def obtener_html(url):
    with requests.Session() as session:  # Utiliza una sesión para reutilizar la conexión
        respuesta = session.get(url)
        respuesta.raise_for_status()  # Lanzar error si la solicitud falla
        return BeautifulSoup(respuesta.content, 'html.parser')

# Función para extraer la lista de diputados y sus URLs
def extraer_lista_diputados(soup):
    diputados = []
    for enlace in soup.select('a.linkVerde'):
        nombre = enlace.get_text(strip=True)
        url_diputado = 'https://sitllxiii.diputados.gob.mx/' + enlace['href']
        diputados.append({'nombre': nombre, 'url': url_diputado})
    return diputados

# Función para extraer la información educativa de la página de un diputado
def extraer_informacion_educativa(soup):
    educacion = []
    # Encuentra todas las entradas después del encabezado 'ESCOLARIDAD'
    seccion_escolaridad = soup.find('td', class_='TitulosVerde', text='ESCOLARIDAD')
    if seccion_escolaridad:
        # La sección de escolaridad está en la misma tabla, pero después del título 'ESCOLARIDAD'
        entradas_escolaridad = seccion_escolaridad.find_all_next('tr')
        for entrada in entradas_escolaridad:
            celdas = entrada.find_all('td', class_='textoNegro')
            # Podría haber filas que no pertenecen a la sección escolaridad, como 'TRAYECTORIA POLÍTICA'
            # Así que verifica si la fila pertenece a la sección escolaridad
            if celdas and len(celdas) >= 2:  # Asumiendo que hay al menos dos celdas (grado y campo)
                # Extraer el grado, campo y periodo si están presentes
                grado = celdas[0].get_text(strip=True) if len(celdas) > 0 else ''
                campo = celdas[1].get_text(strip=True) if len(celdas) > 1 else ''
                periodo = celdas[2].get_text(strip=True) if len(celdas) > 2 else ''
                educacion.append({'grado': grado, 'campo': campo, 'periodo': periodo})
            # Si se encuentra un nuevo encabezado, se detiene la extracción
            if entrada.find('td', class_='TitulosVerde'):
                break
    return educacion

# Función para extraer los enlaces a las iniciativas de un diputado
def extraer_enlaces_iniciativas(soup):
    enlace_iniciativas = soup.find('a', text='Iniciativas')
    if enlace_iniciativas:
        url_iniciativas = 'https://sitllxiii.diputados.gob.mx/' + enlace_iniciativas['href']
        soup_iniciativas = obtener_html(url_iniciativas)
        enlaces_periodos = soup_iniciativas.select('a.linkVerde')
        iniciativas = [extraer_texto_iniciativas('https://sitllxiii.diputados.gob.mx/' + enlace['href']) for enlace in enlaces_periodos]
        return iniciativas
    return []

# Función para extraer el texto de las iniciativas de un periodo de sesiones
def extraer_texto_iniciativas(url_periodo):
    soup_periodo = obtener_html(url_periodo)
    textos = []
    for iniciativa in soup_periodo.select('span.Estilo71'):
        texto = iniciativa.get_text(strip=True)
        if texto:
            textos.append(texto)
    return textos

# Función para obtener la información de un diputado
def obtener_info_diputado(url):
    soup_diputado = obtener_html(url)
    nombre = soup_diputado.find('span', class_='Estilo67').text.strip()
    educacion = extraer_informacion_educativa(soup_diputado)
    iniciativas = extraer_enlaces_iniciativas(soup_diputado)
    entidad = soup_diputado.find('span', text='Entidad:').find_next('td').text.strip()
    return {
        'nombre': nombre,
        'entidad': entidad,
        'educacion': educacion,
        'iniciativas': iniciativas
    }

# Función principal para extraer la información de todos los diputados
def extraer_informacion_diputados_concurrent():
    soup_lista = obtener_html(url_base)
    lista_diputados = extraer_lista_diputados(soup_lista)
    urls_diputados = [diputado['url'] for diputado in lista_diputados]
    
    datos_diputados = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(obtener_info_diputado, url): url for url in urls_diputados}
        for future in as_completed(future_to_url):
            try:
                datos_diputados.append(future.result())
            except Exception as exc:
                print(f'Hubo un problema con la URL {future_to_url[future]}: {exc}')
    
    return pd.DataFrame(datos_diputados)

# Ejecutar la función principal y mostrar/guardar el DataFrame resultante
df_diputados = extraer_informacion_diputados_concurrent()
print(df_diputados)

In [6]:
# Suponiendo que 'df_diputados' es tu DataFrame

# Convertir el DataFrame a CSV
nombre_archivo = "diputados_1.3.csv"
df_diputados.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

print(f"Archivo guardado como: {nombre_archivo}")

Archivo guardado como: diputados_1.3.csv


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# URL de la página con la lista de diputados
url_base = 'https://sitllxiii.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Función para obtener el objeto BeautifulSoup de una página de forma concurrente
def obtener_html(url):
    with requests.Session() as session:  # Utiliza una sesión para reutilizar la conexión
        respuesta = session.get(url)
        respuesta.raise_for_status()  # Lanzar error si la solicitud falla
        return BeautifulSoup(respuesta.content, 'html.parser')

# Función para extraer la lista de diputados y sus URLs
def extraer_lista_diputados(soup):
    diputados = []
    for enlace in soup.select('a.linkVerde'):
        nombre = enlace.get_text(strip=True)
        url_diputado = 'https://sitllxiii.diputados.gob.mx/' + enlace['href']
        diputados.append({'nombre': nombre, 'url': url_diputado})
    return diputados

# Función para extraer la información educativa de la página de un diputado
def extraer_informacion_educativa(soup):
    educacion = []
    # Encuentra todas las entradas después del encabezado 'ESCOLARIDAD'
    seccion_escolaridad = soup.find('td', class_='TitulosVerde', text='ESCOLARIDAD')
    if seccion_escolaridad:
        # La sección de escolaridad está en la misma tabla, pero después del título 'ESCOLARIDAD'
        entradas_escolaridad = seccion_escolaridad.find_all_next('tr')
        for entrada in entradas_escolaridad:
            celdas = entrada.find_all('td', class_='textoNegro')
            # Podría haber filas que no pertenecen a la sección escolaridad, como 'TRAYECTORIA POLÍTICA'
            # Así que verifica si la fila pertenece a la sección escolaridad
            if celdas and len(celdas) >= 2:  # Asumiendo que hay al menos dos celdas (grado y campo)
                # Extraer el grado, campo y periodo si están presentes
                grado = celdas[0].get_text(strip=True) if len(celdas) > 0 else ''
                campo = celdas[1].get_text(strip=True) if len(celdas) > 1 else ''
                periodo = celdas[2].get_text(strip=True) if len(celdas) > 2 else ''
                educacion.append({'grado': grado, 'campo': campo, 'periodo': periodo})
            # Si se encuentra un nuevo encabezado, se detiene la extracción
            if entrada.find('td', class_='TitulosVerde'):
                break
    return educacion

# Función para extraer los enlaces a las iniciativas de un diputado
def extraer_enlaces_iniciativas(soup):
    enlace_iniciativas = soup.find('a', text='Iniciativas')
    if enlace_iniciativas:
        url_iniciativas = 'https://sitllxiii.diputados.gob.mx/' + enlace_iniciativas['href']
        soup_iniciativas = obtener_html(url_iniciativas)
        enlaces_periodos = soup_iniciativas.select('a.linkVerde')
        iniciativas = [extraer_texto_iniciativas('https://sitllxiii.diputados.gob.mx/' + enlace['href']) for enlace in enlaces_periodos]
        return iniciativas
    return []

# Función para extraer el texto de las iniciativas de un periodo de sesiones
def extraer_texto_iniciativas(url_periodo):
    soup_periodo = obtener_html(url_periodo)
    textos = []
    for iniciativa in soup_periodo.select('span.Estilo71'):
        texto = iniciativa.get_text(strip=True)
        if texto:
            textos.append(texto)
    return textos

# Función para extraer la entidad de la página de un diputado
def extraer_entidad(soup):
    # Intenta encontrar el contenedor específico que incluye la entidad
    # buscando por texto y considerando la estructura general
    contenedor_entidad = soup.find("td", text=lambda x: x and "Entidad:" in x)
    
    if contenedor_entidad:
        # Buscar el siguiente elemento hermano que contendría el nombre de la entidad
        entidad = contenedor_entidad.find_next_sibling("td")
        if entidad:
            # Extraer y limpiar el texto de la entidad
            texto_entidad = entidad.get_text(strip=True)
            return texto_entidad
    return "No disponible"  # Retorna esto si no se encuentra la entidad

# Función para obtener la información completa de un diputado
def obtener_info_diputado(url):
    soup_diputado = obtener_html(url)
    
    # Extraer el nombre del diputado
    elemento_nombre = soup_diputado.find('font', face='Arial, Helvetica, sans-serif')
    nombre = elemento_nombre.get_text(strip=True) if elemento_nombre else 'Nombre no disponible'
    
    # Extraer la entidad del diputado
    entidad = extraer_entidad(soup_diputado)
    educacion = extraer_informacion_educativa(soup_diputado)
    iniciativas = extraer_enlaces_iniciativas(soup_diputado)
    return {
        'nombre': nombre,
        'entidad': entidad,
        'educacion': educacion,
        'iniciativas': iniciativas
    }

# Función principal para extraer la información de todos los diputados
def extraer_informacion_diputados_concurrent():
    soup_lista = obtener_html(url_base)
    lista_diputados = extraer_lista_diputados(soup_lista)
    urls_diputados = [diputado['url'] for diputado in lista_diputados]
    
    datos_diputados = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(obtener_info_diputado, url): url for url in urls_diputados}
        for future in as_completed(future_to_url):
            try:
                diputado_info = future.result()
                if diputado_info:
                    datos_diputados.append(diputado_info)
            except Exception as exc:
                print(f'Hubo un problema con la URL {future_to_url[future]}: {exc}')
    
    return pd.DataFrame(datos_diputados)

# Ejecutar la función principal y mostrar/guardar el DataFrame resultante
df_diputados = extraer_informacion_diputados_concurrent()
print(df_diputados)

Hubo un problema con la URL https://sitllxiii.diputados.gob.mx/./curricula.php?dipt=200: HTTPSConnectionPool(host='sitllxiii.diputados.gob.mx', port=443): Max retries exceeded with url: /iniciativas_por_pernplxiii.php?iddipt=200&pert=9 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001DCA005C350>, 'Connection to sitllxiii.diputados.gob.mx timed out. (connect timeout=None)'))
Hubo un problema con la URL https://sitllxiii.diputados.gob.mx/./curricula.php?dipt=104: HTTPSConnectionPool(host='sitllxiii.diputados.gob.mx', port=443): Max retries exceeded with url: /iniciativas_por_pernplxiii.php?iddipt=104&pert=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001DC9F593410>, 'Connection to sitllxiii.diputados.gob.mx timed out. (connect timeout=None)'))
Hubo un problema con la URL https://sitllxiii.diputados.gob.mx/./curricula.php?dipt=420: HTTPSConnectionPool(host='sitllxiii.diputados.gob.mx', port=443): Max retries exceede

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# URL de la página con la lista de diputados
url_base = 'https://sitllxiii.diputados.gob.mx/listado_diputados_gpnp.php?tipot=TOTAL'

# Función para obtener el objeto BeautifulSoup de una página de forma concurrente
def obtener_html(url):
    with requests.Session() as session:  # Utiliza una sesión para reutilizar la conexión
        respuesta = session.get(url)
        respuesta.raise_for_status()  # Lanzar error si la solicitud falla
        return BeautifulSoup(respuesta.content, 'html.parser')

# Función para extraer la lista de diputados y sus URLs
def extraer_lista_diputados(soup):
    diputados = []
    for enlace in soup.select('a.linkVerde'):
        nombre = enlace.get_text(strip=True)
        url_diputado = 'https://sitllxiii.diputados.gob.mx/' + enlace['href']
        diputados.append({'nombre': nombre, 'url': url_diputado})
    return diputados

# Función para extraer la información educativa de la página de un diputado
def extraer_informacion_educativa(soup):
    educacion = []
    # Encuentra todas las entradas después del encabezado 'ESCOLARIDAD'
    seccion_escolaridad = soup.find('td', class_='TitulosVerde', text='ESCOLARIDAD')
    if seccion_escolaridad:
        # La sección de escolaridad está en la misma tabla, pero después del título 'ESCOLARIDAD'
        entradas_escolaridad = seccion_escolaridad.find_all_next('tr')
        for entrada in entradas_escolaridad:
            celdas = entrada.find_all('td', class_='textoNegro')
            # Podría haber filas que no pertenecen a la sección escolaridad, como 'TRAYECTORIA POLÍTICA'
            # Así que verifica si la fila pertenece a la sección escolaridad
            if celdas and len(celdas) >= 2:  # Asumiendo que hay al menos dos celdas (grado y campo)
                # Extraer el grado, campo y periodo si están presentes
                grado = celdas[0].get_text(strip=True) if len(celdas) > 0 else ''
                campo = celdas[1].get_text(strip=True) if len(celdas) > 1 else ''
                periodo = celdas[2].get_text(strip=True) if len(celdas) > 2 else ''
                educacion.append({'grado': grado, 'campo': campo, 'periodo': periodo})
            # Si se encuentra un nuevo encabezado, se detiene la extracción
            if entrada.find('td', class_='TitulosVerde'):
                break
    return educacion

# Función para extraer los enlaces a las iniciativas de un diputado
def extraer_enlaces_iniciativas(soup):
    enlace_iniciativas = soup.find('a', text='Iniciativas')
    if enlace_iniciativas:
        url_iniciativas = 'https://sitllxiii.diputados.gob.mx/' + enlace_iniciativas['href']
        soup_iniciativas = obtener_html(url_iniciativas)
        enlaces_periodos = soup_iniciativas.select('a.linkVerde')
        iniciativas = [extraer_texto_iniciativas('https://sitllxiii.diputados.gob.mx/' + enlace['href']) for enlace in enlaces_periodos]
        return iniciativas
    return []

# Función para extraer el texto de las iniciativas de un periodo de sesiones
def extraer_texto_iniciativas(url_periodo):
    soup_periodo = obtener_html(url_periodo)
    textos = []
    for iniciativa in soup_periodo.select('span.Estilo71'):
        texto = iniciativa.get_text(strip=True)
        if texto:
            textos.append(texto)
    return textos

# Función para extraer la entidad de la página de un diputado
def extraer_entidad(soup):
    try:
        entidad_texto = soup.find(text=lambda text: "Entidad:" in text).find_next()
        entidad = entidad_texto.get_text(strip=True) if entidad_texto else "No disponible"
        return entidad.split("|")[0].strip()
    except AttributeError:
        return "No disponible"


# Función para obtener la información completa de un diputado
def obtener_info_diputado(url):
    soup_diputado = obtener_html(url)
    
    # Corrección en la extracción del nombre del diputado
    elemento_nombre = soup_diputado.find('strong', text=lambda t: "Dip." in t)
    nombre = elemento_nombre.get_text(strip=True) if elemento_nombre else 'Nombre no disponible'
    
    entidad = extraer_entidad(soup_diputado)
    
    # Las siguientes líneas se mantienen sin cambios
    educacion = extraer_informacion_educativa(soup_diputado)
    iniciativas = extraer_enlaces_iniciativas(soup_diputado)
    
    return {
        'nombre': nombre,
        'entidad': entidad,
        'educacion': educacion,
        'iniciativas': iniciativas
    }

def extraer_informacion_diputados_concurrent():
    soup_lista = obtener_html(url_base)
    lista_diputados = extraer_lista_diputados(soup_lista)
    urls_diputados = [diputado['url'] for diputado in lista_diputados]
    
    datos_diputados = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(obtener_info_diputado, url): url for url in urls_diputados}
        for future in as_completed(future_to_url):
            try:
                diputado_info = future.result()
                if diputado_info:
                    datos_diputados.append(diputado_info)
            except Exception as exc:
                print(f'Hubo un problema con la URL {future_to_url[future]}: {exc}')
    
    return pd.DataFrame(datos_diputados)

# Ejecutar la función principal y mostrar/guardar el DataFrame resultante
df_diputados = extraer_informacion_diputados_concurrent()
print(df_diputados)

                                   nombre       entidad  \
0         Dip. Erubiel Lorenzo Alonso Que       Tabasco   
1    Dip. María Guadalupe Alcántara Rojas        México   
2          Dip. Antonio Tarek Abdala Saad      Veracruz   
3       Dip. Edith Anabel Alvarado Varela      Tlaxcala   
4               Dip. Fidel Almanza Monroy        México   
..                                    ...           ...   
495           Dip. Javier Guerrero García      Coahuila   
496              Dip. Adriana Sarur Torre      Veracruz   
497          Dip. José Luis Toledo Medina  Quintana Roo   
498          Dip. Francisco Martínez Neri        Oaxaca   
499            Dip. Edgar Spinoso Carrera      Veracruz   

                                             educacion  \
0                                                   []   
1    [{'grado': 'Licenciatura', 'campo': 'Contadurí...   
2    [{'grado': 'Licenciatura', 'campo': 'Prensa Es...   
3    [{'grado': 'Maestría', 'campo': 'Políticas Púb...   
4

In [3]:
# Suponiendo que 'df_diputados' es tu DataFrame

# Convertir el DataFrame a CSV
nombre_archivo = "diputados_1.4.csv"
df_diputados.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

print(f"Archivo guardado como: {nombre_archivo}")

Archivo guardado como: diputados_1.4.csv
